### Install Packages

In [152]:
!pip install scikit-learn
!pip install pandas
!pip install nltk

You should consider upgrading via the 'c:\users\jomah\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.

You should consider upgrading via the 'c:\users\jomah\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.

You should consider upgrading via the 'c:\users\jomah\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.



In [153]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import pickle
import json
import pandas as pd
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from bs4 import BeautifulSoup
from sklearn.pipeline import make_pipeline

import nltk
import textstat
from empath import Empath

### Setup Cleaning Process

In [154]:
stopword_list = set(stopwords.words('german'))
stemmer = SnowballStemmer("english")

def clean_text(text):
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopword_list)
    text = ' '.join(stemmer.stem(word) for word in text.split())
    return text

### Select Content that should be used to classify data

In [155]:

# we need to choose the data that can be used to classify the data
data_for_classification = [["content","extracted_data/"+"content_trafilatura_big_stemmed.json"],["title","extracted_data/"+"title_big_stemmed.json"]]

# the data that was chosen to be used as training data will be stored in this variable called "used_data"
# 0 = title of the websites  // 1 = content of the websites
used_data = data_for_classification[0]
undersampling = False
oversampling = True

'''
Welche Parameter nutzen?

Für Content: [0], undersampling = False, oversampling = True
Für Title: [1], undersampling = False, oversampling = True


'''

'\nWelche Parameter nutzen?\n\nFür Content: [0], undersampling = False, oversampling = True\nFür Title: [1], undersampling = False, oversampling = True\n\n\n'

### Load assessments-Text-File

In [156]:
# load assessments
file_path = "assessments_with_bl.txt"
with open(file_path) as f:
    assessments = f.readlines()
print(len(assessments))

4470


In [157]:
# transform assessments into an array
assessments = [x.strip() for x in assessments]

# delete header
assessments.pop(0)

# store assessments, that do not occur more than once
assessments_without_dublicates = []

# store ids that are allready in assessments_without_dublicates
doc_ids = set()

# fill assessments_without_dublicates
for doc in assessments:
  if (doc.split(' ')[2] in doc_ids) is False:
     assessments_without_dublicates.append(doc)
     doc_ids.add(doc.split(' ')[2])

# so finally after this iteration the assessments array does only consist of individual documents
assessments = assessments_without_dublicates

# we don't want to use the test data for training 
# so we need to delete these documents from the training data
with open("test_document_ids.txt","r") as f:
    test_document_ids = [line.rstrip() for line in f]

# we will store the ids of the assessments that are also part of the training data
duplicate_indexes = []

# iteration to see which ids of the assessments are also part of the training data
i = 0
for d in assessments:
  for igd in test_document_ids:
    if d.split(' ')[2] == igd:
       duplicate_indexes.append(i)
  i += 1

# each assessment that is not part of the training data will now be stored in this array
assessments_with_removal = []

# iteration to see which assessment  is not part of the training data
for i in range(len(assessments)):
    if not i in duplicate_indexes:
        assessments_with_removal.append(assessments[i])

# so finally after this iteration the assessments array does only consist of documents that aren't used for testing
assessments = assessments_with_removal

print(len(assessments))

4092


### Create Arrays with correct and not correct documents

In [158]:
correct_documents = []
not_correct_documents = []

# load the content data
with open(used_data[1],encoding="utf-8") as json_file:
    content_data = json.load(json_file)

# create index variable to print progress in console
index = 0

# add the content of each assessment either to the array of correct_documents or not_correct_documents
for doc in assessments:
  index += 1
  try:
    split_doc = doc.split(' ')

    id_ = split_doc[2]

    # extract the content of an id
    content = content_data[id_]

    # optional:
    #content = clean_text(content)

    # store the correctness assessment in a variable
    correctness = split_doc[3]

    # store the content in one of the two arrays
    if correctness == "2" and content != None and correctness!="0": # yes
        correct_documents.append((id_,content))
    if correctness == "1" and content != None and correctness!="0": # no
        not_correct_documents.append((id_,content))
    if correctness != "0" and content != None:
        print("added: ",index, id_, correctness)
    else:
        print("error: ", index, id_, correctness)
  except:
    print("error: ",id_)
    pass

print(len(correct_documents), len(not_correct_documents))

error:  1 00133722-4dfd-484c-9cda-580f176d4cbd 0
error:  2 00b10eb6-cdae-4c98-9468-edbfc452eef6 0
error:  3 00f5d8ec-bfb5-40c0-942e-82d8fb0fb4b8 0
error:  4 0161242a-3a47-4c59-91e6-f16661a79c39 0
error:  5 020e21e9-78cd-4cd3-9684-097f97791540 0
error:  6 022ab0a8-01fe-4fc9-87e8-080c9f90e9ec 0
error:  7 02f07c83-f0a7-42d1-a634-a183137bf7ab 0
error:  8 02f1ad99-4d44-4c4e-a345-ebe0f1abf4a7 0
error:  9 0434ffc3-8bfb-4f9f-883f-da0b4a04b762 0
error:  10 04beca6c-a387-48db-b9d4-b53aa86439ae 0
error:  11 053d23aa-741b-4ad6-a053-2affd3a04af6 0
error:  12 058e62cd-0a39-4c61-9626-d00d31e4a647 0
error:  13 067facd4-28a1-4d87-9400-fa290bfd11c8 0
error:  14 07f2d394-5beb-4b56-95bd-01e9d2e5a998 0
added:  15 08126c59-1203-4eae-8349-0ce70836ac10 1
error:  16 096285c2-4e7c-46bd-8942-8382321b2144 0
error:  17 0a6b5bca-b37f-4238-a86a-15b54da72848 0
error:  18 0a8f4df5-e9a1-4327-902a-4fc8ef9882d7 0
error:  19 0ad04b2b-2bd2-4308-8d64-2c2a7dc4bc84 0
error:  20 0b2551c8-6f80-41da-9535-7d7d9d17f2ac 0
error:  2

### We also need to save documents without oversampling / undersampling

In [159]:
# We will now create two arrays: one for the labels of each content and one with the content itself
labeled_data_raw = []

for content in not_correct_documents:
    labeled_data_raw.append([content[0],content[1],"fake"])
for content in correct_documents:
    labeled_data_raw.append([content[0],content[1],"real"])

df_raw = pd.DataFrame.from_records(labeled_data_raw)

# After we created these pairs, we can store them in normal arrays
raw_txt = df_raw[1].tolist()
labels_raw = df_raw[2].tolist()

### Set Oversampling / Undersampling Settings 

In [160]:
def balance_sample(undersampling=False, oversampling=False):
    if undersampling == True:
        while len(correct_documents) != len(not_correct_documents):
            if len(correct_documents) > len(not_correct_documents):
                correct_documents.pop(random.randrange(len(correct_documents)))
            else:
                not_correct_documents.pop(random.randrange(len(not_correct_documents)))
        print("Amount of not correct | correct data after undersampling: ", len(not_correct_documents), len(correct_documents))
    if oversampling == True:
        while len(correct_documents)!=len(not_correct_documents):
            if len(not_correct_documents) < len(correct_documents):
                not_correct_documents.append(random.choice(not_correct_documents))
            else:
                correct_documents.append(random.choice(correct_documents))
        print("Amount of not correct | correct data after oversampling: ", len(not_correct_documents), len(correct_documents))

balance_sample(undersampling,oversampling)

Amount of not correct | correct data after oversampling:  2052 2052


### Convert these two arrays into data that is understandable for sklearn (Pandas Array)

In [161]:
# We will now create two arrays: one for the labels of each content and one with the content itself
labeled_data = []

for content in not_correct_documents:
    labeled_data.append([content[0],content[1],"fake"])
for content in correct_documents:
    labeled_data.append([content[0],content[1],"real"])

df = pd.DataFrame.from_records(labeled_data)

# After we created these pairs, we can store them in normal arrays
ids = df[0].tolist()
txt = df[1].tolist()
labels = df[2].tolist()

### Evaluate a classifier trained on the training data (5 Folds)

In [162]:
def list_to_count_dict(list_):
    counts = dict()
    for i in list_:
        counts[i] = counts.get(i,0) + 1
    return counts

def pos_tag_text(text):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def words_and_pos_tags_to_pos_tags_only(words_and_pos_tags):
    return [i[1] for i in words_and_pos_tags]

def pos_tags_only_to_counted_pos_tags(pos_tags_only):
    counts = dict()
    for i in pos_tags_only:
        counts[i] = counts.get(i,0) + 1
    return counts

# use this
def text_to_pos_tag_counts(text):
    words_and_pos_tags = pos_tag_text(text)
    pos_tags = words_and_pos_tags_to_pos_tags_only(words_and_pos_tags)
    pos_tags_only_to_counted_pos_tags = list_to_count_dict(pos_tags)
    return pos_tags_only_to_counted_pos_tags

# use this
def text_to_empath_analysis(text):
    lexicon = Empath()
    return lexicon.analyze(text)

# use this
def text_to_flesch_reading_ease(text):
    return textstat.flesch_reading_ease(text)

# use this
def soup_to_tag_count(soup):
    html_tags = [tag.name for tag in soup.find_all()]
    return list_to_count_dict(html_tags)

In [163]:
pos_tag_counts = [text_to_pos_tag_counts(t) for t in txt]

In [164]:
flesch_reading_eases = [text_to_flesch_reading_ease(t) for t in txt]

In [165]:
empath_analysis = [text_to_empath_analysis(t) for t in txt]

In [166]:
html_tags = []
for id_ in ids:
    try:
        with open("documents/"+id_,encoding="utf-8") as f:
            html_tags.append(soup_to_tag_count(BeautifulSoup(f.read(),"html.parser")))
    except:
        print("error",id_)
        html_tags.append(None)

In [167]:
pos_tags = []
for tags in pos_tag_counts:
    for key in tags:
        if key not in pos_tags:
            pos_tags.append(key)
html_tag_list = []
for html_tag in html_tags:
    for key in html_tag:
        if key not in html_tag_list:
            html_tag_list.append(key)
empath_cats = list(empath_analysis[0].keys())

In [168]:
to_df_dict = dict()

to_df_dict["doc_id"] = ids
to_df_dict["labels"] = labels

for tag in pos_tags:
    for doc in pos_tag_counts:
        if tag in doc:
            if "pos_"+tag in to_df_dict:
                to_df_dict["pos_"+tag].append(doc[tag])
            else:
                to_df_dict["pos_"+tag] = [doc[tag]]
        else:
            if "pos_"+tag in to_df_dict:
                to_df_dict["pos_"+tag].append(0)
            else:
                to_df_dict["pos_"+tag] = [0]

to_df_dict["flesch_reading_ease"] = flesch_reading_eases

for cat in empath_cats:
    for doc in empath_analysis:
        if cat in doc:
            if "empath_"+cat in to_df_dict:
                to_df_dict["empath_"+cat].append(doc[cat])
            else:
                to_df_dict["empath_"+cat] = [doc[cat]]

for tag in html_tag_list:
    for doc in html_tags:
        if tag in doc:
            if "html_"+tag in to_df_dict:
                to_df_dict["html_"+tag].append(doc[tag])
            else:
                to_df_dict["html_"+tag] = [doc[tag]]
        else:
            if "html_"+tag in to_df_dict:
                to_df_dict["html_"+tag].append(0)
            else:
                to_df_dict["html_"+tag] = [0]

In [169]:
df = pd.DataFrame.from_dict(to_df_dict)
df

,doc_id,labels,pos_NN,pos_PRP,pos_IN,pos_DT,pos_JJ,pos_TO,pos_VB,pos_CD,...,html_uc-sidebar-block,html_uc-social-embed,html_twitter-widget,html_dofollow,html_fb:post,html_w-ad,html_esi:remove,html_esi:include,html_keyword,html_h77
0,08126c59-1203-4eae-8349-0ce70836ac10,fake,206,15,68,57,53,13,19,11,...,0,0,0,0,0,0,0,0,0,0
1,0f766b1c-14b6-4954-84bf-b12d25614c87,fake,85,4,13,15,30,11,8,12,...,0,0,0,0,0,0,0,0,0,0
2,12667418-1647-41d0-8878-e5e3b60bbb73,fake,375,29,133,149,127,38,67,7,...,0,0,0,0,0,0,0,0,0,0
3,155e0f30-0866-46be-9bda-12132263448e,fake,200,58,71,54,63,20,40,15,...,0,0,0,0,0,0,0,0,0,0
4,204bee2b-cd4c-4183-b8a8-de50401ad8e5,fake,61,4,11,13,23,9,6,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,eda64168-e7f5-4f7c-bf29-18ed7b09a7da,real,27,0,11,9,7,0,4,1,...,0,0,0,0,0,0,0,0,0,0
4100,efed3162-3809-45e6-9fe8-f58c888b777b,real,94,9,31,24,26,13,15,9,...,0,0,0,0,0,0,0,0,0,0
4101,f52ef7f8-314c-4a64-97fe-377dec2c9142,real,733,55,249,239,239,57,111,54,...,0,0,0,0,0,0,0,0,0,0
4102,fa9fd78d-8160-441f-acda-92365cdad997,real,573,57,215,212,192,52,96,33,...,0,0,0,0,0,0,0,0,0,0


In [170]:
feature_names = list(df.columns)
feature_names.remove('doc_id')
feature_names.remove('labels')

In [171]:
X = df.loc[:, feature_names]
y = df.labels

In [172]:
# Used classifier: SVC
classifier = make_pipeline(StandardScaler(),svm.SVC(decision_function_shape='ovo',probability=True))
classifier.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(decision_function_shape='ovo', probability=True))])

In [173]:
cross_val = cross_val_score(classifier,X,y,cv=5)
print(cross_val)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val.mean(), cross_val.std() * 2))

[0.68331303 0.61266748 0.57612667 0.58587089 0.74878049]
Accuracy: 0.64 (+/- 0.13)


In [174]:
raw_txt = np.array(raw_txt)
labels_raw = np.array(labels_raw)
print(len(raw_txt),len(labels_raw))

3335 3335


### Store results of classification

In [175]:
# topics of the documents in correct order
topics_y = [9,9,9,9,9,9,9,9,20,20,20,20,20,20,20,20,25,25,25,25,25,25,25,25,30,30,30,30,30,30,30,30,47,47,47,47,47,47,47,47,49,49,49,49,49,49,49,49]

# correct labels of the test_data
test_y = ["real","real","fake","real","real","fake","fake","fake","fake","fake","real","fake","fake","real","real","real","real","fake","fake","real","real","fake","fake","real","real","real","real","fake","fake","fake","real","fake","fake","real","real","fake","fake","real","real","fake","fake","fake","real","real","fake","fake","real","real"]

test_X = []
for doc in test_document_ids:
    cont = content_data[doc]
    pos = text_to_pos_tag_counts(cont)
    em = text_to_empath_analysis(cont)
    fle = text_to_flesch_reading_ease(cont)
    with open("documents/"+doc,encoding="utf-8") as f:
        htm = soup_to_tag_count(BeautifulSoup(f.read(),"html.parser"))
    to_df_dict_ = {}

    for tag in pos_tags:
        if tag in pos:
            to_df_dict_["pos_"+tag] = [pos[tag]]
        else:
            to_df_dict_["pos_"+tag] = [0]
    to_df_dict_["flesch_reading_ease"] = [fle]
    for tag in empath_cats:
        if tag in em:
            to_df_dict_["empath_"+tag] = [em[tag]]
        else:
            to_df_dict_["empath_"+tag] = [0]
    for tag in html_tag_list:
        if tag in htm:
            to_df_dict_["html_"+tag] = [htm[tag]]
        else:
            to_df_dict_["html_"+tag] = [0]
    test_X.append((doc,pd.DataFrame.from_dict(to_df_dict_)))


# these variables are used to calculate the accuracy
index = 0
correct = 0

# this variable will be used to store the predictions to export it to json
predictions = {}

for x in test_X:
    id_ = test_X[index][0]
    data = test_X[index][1]
    pred = classifier.predict(data)[0]
    cer = max(classifier.predict_proba(data).tolist()[0])
    print(pred,cer,pred == test_y[index])

    predictions[id_] = {"topic":topics_y[index],"prediction":pred,"certainity":cer}

    if pred == test_y[index]:
        correct += 1
    index += 1

# for x in test_X:
#     print("ass: / cltf: ",test_y[index],classifier.predict(tfidfconverter.transform([x]).toarray())[0],"clf_pr: ",max(classifier.predict_proba(tfidfconverter.transform([x]).toarray()).tolist()[0]))
#     predictions[test_document_ids[index]] = {"topic":topics_y[index],"prediction":classifier.predict(tfidfconverter.transform([x]).toarray())[0],"certainity": max(classifier.predict_proba(tfidfconverter.transform([x]).toarray().tolist())[0])}
#     if classifier.predict(tfidfconverter.transform([x]).toarray())[0] == test_y[index]:
#         correct += 1
#     index += 1

print(correct/index)

with open("predictions/topic_agonistic_predictions"+".json", 'w') as outfile:
    json.dump(predictions, outfile)

fake 0.5155686037926317 False
fake 0.5071073129256548 False
fake 0.8453706122684108 True
real 0.6050220217583967 True
fake 0.7532441225299578 False
fake 0.6310529685901469 True
fake 0.5967087713101957 True
fake 0.6961454564732512 True
real 0.685265730424305 False
real 0.6050144666541099 False
real 0.8275896341376123 True
fake 0.8675154070856227 True
fake 0.5 True
real 0.725200470750714 True
real 0.9035096846517658 True
real 0.9577663015714287 True
real 0.6763215954761309 True
fake 0.6318582076549047 True
fake 0.7318861457861228 True
real 0.9374592660331018 True
real 0.79964354437004 True
fake 0.5711913699581579 True
fake 0.656907871810212 True
real 0.9391874557477857 True
real 0.517789374023596 True
fake 0.8128598266639995 False
fake 0.7160295171688615 False
fake 0.8294655269440926 True
fake 0.5920277194125144 True
real 0.552529164386187 False
real 0.875406181852939 True
fake 0.5 True
fake 0.7595728105055277 True
real 0.6454251934930321 True
real 0.7846559855325992 True
fake 0.51718250